# 0. Install Dependencies

In [2]:
# import tensorflow
# tensorflow.__version__

'2.3.1'

In [18]:
!pip install tensorflow-gpu==2.3.1
!pip install gym
!pip install keras
!pip install keras-rl2

# 1. Test Random Environment with OpenAI Gym

In [1]:
import gym
import random
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.ppo.policies import MlpPolicy

from mlagents_envs.environment import UnityEnvironment
from gym_unity.envs import UnityToGymWrapper
from stable_baselines3.common.vec_env.subproc_vec_env import SubprocVecEnv
from stable_baselines3.common.vec_env.dummy_vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor
# from baselines import logger
# import baselines.ppo2.ppo2 as ppo2

import os
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel

In [42]:
try:
    env.close()
except:
    pass

def test():
    rank = 0
    # create unity env
    channel = EngineConfigurationChannel()
    unity_env = UnityEnvironment("run31_noBrain", base_port=5000, side_channels=[channel])
    channel.set_configuration_parameters(time_scale = 20.0)
    # unity_wrapper
    env = UnityToGymWrapper(unity_env, uint8_visual=False, flatten_branched=True, allow_multiple_obs=True) #
    # add monitor
    env = Monitor(env)
    # vectorize environment
    return env
env = test()

[INFO] Connected to Unity environment with package version 2.0.0-exp.1 and communication version 1.5.0
[INFO] Connected new brain: Hummingbird?team=0


In [43]:
env.metadata

{'render.modes': ['rgb_array']}

In [44]:
env.reward_range

(-inf, inf)

In [45]:
env.observation_space

Tuple(Box(-inf, inf, (45,), float32))

In [46]:
env.action_space

Discrete(27)

In [48]:
env.close()

## Vectorized Environment

In [8]:
venv = DummyVecEnv([lambda: env])
venv

In [24]:
# states = venv.observation_space.shape[0] <-- doesnt work, seems like it returns a tuple of shape 1 (x)
states = venv.observation_space[0].shape[0]
actions = venv.action_space.n

In [17]:
venv.observation_space

Tuple(Box(-inf, inf, (45,), float32))

In [19]:
venv.observation_space[0]

Box(-inf, inf, (45,), float32)

In [21]:
type(venv.observation_space[0])

gym.spaces.box.Box

In [26]:
venv.observation_space[0].shape[0]
states

45

In [25]:
actions

27

In [32]:
# gives an error :TypeError: cannot pickle '_thread.lock' object
# doesnt work
# venv = SubprocVecEnv([lambda: env])
# venv

## Vectorized with Formula

In [ ]:
try:
    from mpi4py import MPI
except ImportError:
    MPI = None

def make_unity_env(env_directory, num_env, visual, start_index=0):
    """
    Create a wrapped, monitored Unity environment.
    """
    def make_env(rank, use_visual=True): # pylint: disable=C0111
        print("usevisual:", use_visual)
        def _thunk():
            channel = EngineConfigurationChannel()
            unity_env = UnityEnvironment(env_directory, base_port=5000 + rank, side_channels=[channel])
            channel.set_configuration_parameters(time_scale = 20.0)
            env = UnityToGymWrapper(unity_env, uint8_visual=True, flatten_branched=True, allow_multiple_obs=True) #
            env = Monitor(env)
            return env
        return _thunk
    return DummyVecEnv([make_env(0, visual)])
    if visual:
        return SubprocVecEnv([make_env(i + start_index) for i in range(num_env)])
    else:
        rank = MPI.COMM_WORLD.Get_rank() if MPI else 0
        return DummyVecEnv([make_env(rank, use_visual=False)])

In [ ]:

vnev = make_unity_env("run31_noBrain", 1, False)

In [8]:
env.close()
venv = DummyVecEnv([lambda: env])
    return venv

In [7]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:19.0
Episode:2 Score:15.0
Episode:3 Score:12.0
Episode:4 Score:11.0
Episode:5 Score:67.0
Episode:6 Score:35.0
Episode:7 Score:29.0
Episode:8 Score:13.0
Episode:9 Score:15.0
Episode:10 Score:17.0


# 2. Create a Deep Learning Model with Keras

In [8]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [9]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [10]:
model = build_model(states, actions)

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 4)                 0         
_________________________________________________________________
dense (Dense)                (None, 24)                120       
_________________________________________________________________
dense_1 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [12]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [13]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [21]:
del model
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 112s 11ms/step - reward: 1.0000
105 episodes - episode_reward: 93.733 [9.000, 200.000] - loss: 2.209 - mae: 18.578 - mean_q: 37.607

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 110s 11ms/step - reward: 1.0000
52 episodes - episode_reward: 194.827 [161.000, 200.000] - loss: 5.109 - mae: 38.310 - mean_q: 77.438

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 106s 11ms/step - reward: 1.0000
50 episodes - episode_reward: 196.800 [173.000, 200.000] - loss: 4.735 - mae: 41.417 - mean_q: 83.492

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 114s 11ms/step - reward: 1.0000
51 episodes - episode_reward: 199.745 [187.000, 200.000] - loss: 6.391 - mae: 40.217 - mean_q: 80.806

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 114s 11ms/step - reward: 1.

In [23]:
scores = dqn.test(env, nb_episodes=10, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
200.0


# 4. Reloading Agent from Memory

In [25]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [26]:
del model
del dqn
del env

In [27]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [28]:
dqn.load_weights('dqn_weights.h5f')

In [29]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
